In [1]:
import os

from seml import get_results
import matplotlib.pyplot as plt
import torch
import pandas as pd

import seaborn as sns

In [ ]:
df = get_results('dp_timeseries_eval_pld_label_noise',
                 fields=['config.privacy_loss_kwargs.min_sequence_length',
                         'config.privacy_loss_kwargs.batch_size',
                         'config.privacy_loss_kwargs.future_target_noise_multiplier',
                         'result.log_dir'],
            to_data_frame=True)

df = df.drop(columns=['_id'])

df

In [3]:
columns = df.columns

df = df.rename(columns={
    c: c.split('.')[-1]
    for c in columns
})

In [4]:
def epsilons_deltas_from_log_dir(df):
    data_dict = {
        'epsilons': [],
        'deltas': []}

    for log_dir in df['log_dir']:
        
        save_dict = torch.load(os.path.join(log_dir, 'config_and_results.pyt'))
        data_dict['epsilons'].append(save_dict['results']['epsilons'])
        data_dict['deltas'].append(save_dict['results']['deltas'])
    
    return pd.DataFrame(data_dict)

In [ ]:
df_results = pd.concat((df, epsilons_deltas_from_log_dir(df)), axis=1)
df_results.head()

In [6]:
def plot_tradeoff(df, min_sequence_length, batch_size, noise_multiplier_filter=None, xlim=100):
    df = df.copy()

    df = df.loc[(df['min_sequence_length'] == min_sequence_length)
                & (df['batch_size'] == batch_size)]
    
    if noise_multiplier_filter is not None:
        df = df[df['future_target_noise_multiplier'].isin(noise_multiplier_filter)]

    df = df[['future_target_noise_multiplier', 'epsilons', 'deltas']]
    df = df.sort_values('future_target_noise_multiplier')

    sns.set_theme()
    pal = sns.color_palette("rocket_r", 5)

    fig, ax = plt.subplots(1, 1)
    
    for i, (_, (sigma, epsilons, deltas)) in enumerate(df.iterrows()):

        #deltas = deltas[(0 <= epsilons) & (epsilons <= 1)]
        #epsilons = epsilons[(0 <= epsilons) & (epsilons <= 1)]

        deltas = deltas[epsilons <= xlim]
        epsilons = epsilons[epsilons <= xlim]
        
        if sigma > 1000:
            label = None
            color = pal[0]
            linestyle = 'dashed'
        else:
            label = f'${sigma}$'
            color = pal[i]
            linestyle = 'solid'

        ax.plot(epsilons, deltas, label=label, color=color, linestyle=linestyle, zorder=3, clip_on=False)


    ax.set_xlim(left=epsilons.min(), right=xlim)
    ax.set_ylim(bottom=0)
    ax.set_xscale('log')
    #plt.yscale('log')

    ax.minorticks_off()

    legend_1 = ax.legend(title='$\sigma_F$', title_fontsize=10, loc='upper right')
    ax.set_xlabel('$\epsilon$')
    ax.set_ylabel('$\delta(\epsilon)$')

    # Separate legend for linestyle
    linestyles = ['solid', 'dashed']
    labels = ['$L_C = 60, L_F = 30$', '$L_C = 60, L_F = 0$']
    dummy_lines = []
    for linestyle, label in zip(linestyles, labels):
        dummy_lines.append(ax.plot([],[], c="black", linestyle=linestyle)[0])
    ax.legend(dummy_lines, labels, loc='lower left')

    # Add old legend back
    ax.add_artist(legend_1)

In [7]:
from itertools import product

min_sequence_lengths = [6, 30]
batch_sizes = [32, 160]

noise_multiplier_filter = [0.0, 0.5, 1.0, 2.0, 5.0, 1, 1000000000.0]

save_dir = '/ceph/hdd/staff/schuchaj/dp_timeseries_plots_camera_icml25/eval_pld_label_noise'

for min_sequence_length, batch_size in product(min_sequence_lengths, batch_sizes):

    plot_tradeoff(df_results, min_sequence_length, batch_size, noise_multiplier_filter)

In [8]:
from itertools import product

noise_multiplier_filter = [0.0, 0.5, 1.0, 2.0, 5.0, 1, 1000000000.0]

save_dir = '/ceph/hdd/staff/schuchaj/dp_timeseries_plots_camera_icml25/eval_pld_label_noise'

for min_sequence_length, batch_size in [(30, 32)]:

    plot_tradeoff(df_results, min_sequence_length, batch_size, noise_multiplier_filter, xlim=10)